In [ ]:
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2

In [ ]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [ ]:
import os
import cv2
import glob
import torch
import random
import pydicom
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from scipy.ndimage.interpolation import zoom
from sklearn.model_selection import *

## Init

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results

    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True  # False

In [ ]:
SEED = 2020
seed_everything(SEED)

MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

SIZE = 256

IMG_TARGET = "pe_present_on_image"

EXAM_TARGETS = [
    "negative_exam_for_pe",
    "rv_lv_ratio_gte_1",
    "rv_lv_ratio_lt_1",
    "leftsided_pe",
    "chronic_pe",
    "rightsided_pe",
    "acute_and_chronic_pe",
    "central_pe",
    "indeterminate",
]

NUM_EXAM_TARGETS = len(EXAM_TARGETS)

In [ ]:
DATA_PATH = "../input/rsna-str-pulmonary-embolism-detection/"

## Dicom processing

In [ ]:
def load_dicom(f):
    dicom = pydicom.dcmread(f)
    
    M = float(dicom.RescaleSlope)
    B = float(dicom.RescaleIntercept)
    
    z_pos = float(dicom.ImagePositionPatient[-1])
    img = dicom.pixel_array * M + B
    
    return img, z_pos


def window(img, WL=50, WW=350):
    upper, lower = WL + WW // 2, WL - WW // 2
    X = np.clip(img, lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    
    X = (X * 255.0).astype('uint8')
    
    return X

def dicom_to_img(path):
    image, z_pos = load_dicom(path)

    image_lung = window(image, WL=-600, WW=1500)[:, :, np.newaxis]
    image_mediastinal = window(image, WL=40, WW=400)[:, :, np.newaxis]
    image_pe_specific = window(image, WL=100, WW=700)[:, :, np.newaxis]
    
    image = np.concatenate([image_mediastinal, image_pe_specific, image_lung], axis=-1)
    
    rat = SIZE / np.max(image.shape[1:])
    image = zoom(image, [rat, rat,1.], prefilter=False, order=1)
    
    return image, z_pos

In [ ]:
import albumentations as albu 
from albumentations import pytorch as AT


def normalizer(mean=MEAN, std=STD):
    return albu.Compose([
                albu.Normalize(mean=mean, std=std),
                AT.transforms.ToTensor(),
            ],
            p=1,
        )

In [ ]:
def to_jpg(img, name='img'):
    cv2.imwrite(name + ".jpg", img)
    return cv2.imread(name + ".jpg")

In [ ]:
class PatientDataset(Dataset):
    """
    Dataset for feature extraction
    """
    def __init__(self, df, path, transforms=None): 
        self.df = df
        self.path = path
        self.img_paths = [sop + '.dcm' for sop in df['SOPInstanceUID'].values]
        
        assert len(self.img_paths) == len(self.df)
        
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image, z = dicom_to_img(self.path + self.img_paths[idx])
        image = to_jpg(image, name='tmp')
        
        if self.transforms:
            image = self.transforms(image=image)["image"]

        return image, z


# Models

## Utils

In [ ]:
def load_model_weights(model, filename, verbose=1, cp_folder=""):
    """
    Loads the weights of a PyTorch model. The exception handles cpu/gpu incompatibilities

    Arguments:
        model {torch module} -- Model to load the weights to
        filename {str} -- Name of the checkpoint

    Keyword Arguments:
        verbose {int} -- Whether to display infos (default: {1})
        cp_folder {str} -- Folder to load from (default: {''})

    Returns:
        torch module -- Model with loaded weights
    """
    if verbose:
        print(f"\n -> Loading weights from {os.path.join(cp_folder,filename)}\n")
    try:
        model.load_state_dict(os.path.join(cp_folder, filename), strict=False)
    except BaseException:
        model.load_state_dict(
            torch.load(os.path.join(cp_folder, filename), map_location="cpu"),
            strict=True,
        )
    return model

## First level

In [ ]:
import torch
import torch.nn.functional as F


def add_ft_extractor(resnet):
    resnet.extract_ft = lambda x: forward_extract_ft(resnet, x)
    

def forward_extract_ft(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    ft = torch.flatten(x, 1)

    x = self.fc(ft)

    return x.view(-1), ft


def add_ft_extractor_enet(effnet):
    effnet.forward = lambda x: forward_extract_ft_enet(effnet, x)
    

def forward_extract_ft_enet(self, x):
    x = self.extract_features(x)

    x = self._avg_pooling(x)

    ft = x.flatten(start_dim=1)
    x = self._dropout(ft)
    x = self._fc(x)

    return x.view(-1), ft

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import *
from efficientnet_pytorch import EfficientNet


def define_model(name, num_classes=1):
    if "resnext" in name:
        model = resnext50_32x4d(pretrained=False)
        model.nb_ft = model.fc.in_features
        model.fc = nn.Linear(model.nb_ft, num_classes)
        add_ft_extractor(model)
    else:
        model = EfficientNet.from_name(name)
        model.nb_ft = model._fc.in_features
        model._fc = nn.Linear(model.nb_ft, num_classes)
        add_ft_extractor_enet(model)
    
    return model


## 2nd level

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, ft_dim=2048, lstm_dim=64, dense_dim=256, use_msd=False, num_classes=9):
        super().__init__()
        self.use_msd = use_msd
        
        self.mlp = nn.Sequential(
            nn.Linear(ft_dim, dense_dim * 2),
            nn.ReLU(),
            nn.Linear(dense_dim * 2, dense_dim),
            nn.ReLU(),
        )

        self.lstm = nn.LSTM(dense_dim, lstm_dim, batch_first=True, bidirectional=True)
        
        self.logits_exam = nn.Sequential(
            nn.Linear(lstm_dim * 4 + dense_dim * 2, lstm_dim),
            nn.ReLU(),
            nn.Linear(lstm_dim, num_classes),
        )
        
        self.logits_img = nn.Sequential(
            nn.Linear(lstm_dim *  2 + dense_dim, lstm_dim),
            nn.ReLU(),
            nn.Linear(lstm_dim, 1),
        )
        
        self.high_dropout = nn.Dropout(p=0.5)
    
    def forward(self, x):
        features = self.mlp(x)
        features2, _ = self.lstm(features)
        
        features = torch.cat([features, features2], -1)
        
        mean = features.mean(1)
        max_, _ = features.max(1)
        pooled = torch.cat([mean, max_], -1)
        
        if self.use_msd and self.training:
            logits_exam = torch.mean(
                torch.stack(
                    [self.logits_exam(self.high_dropout(pooled)) for _ in range(5)],
                    dim=0,
                    ),
                dim=0,
            )
            
            logits_img = torch.mean(
                torch.stack(
                    [self.logits_img(self.high_dropout(features)) for _ in range(5)],
                    dim=0,
                    ),
                dim=0,
            )
        else:
            logits_exam = self.logits_exam(pooled)
            logits_img = self.logits_img(features)

        return logits_exam, logits_img.squeeze(-1)

## Ensure rules verification

In [ ]:
def post_process(exam_pred, image_pred, eps=1e-6):
    has_pe_image = image_pred.max() > 0.5

    (negative_exam_for_pe, rv_lv_ratio_gte_1, rv_lv_ratio_lt_1, leftsided_pe, 
     chronic_pe, rightsided_pe, acute_and_chronic_pe, central_pe, indeterminate) = exam_pred
    
    broken = False
    
    if has_pe_image:  # 
        if rv_lv_ratio_gte_1 < 0.5 and rv_lv_ratio_lt_1 < 0.5:  # this means image has no PE
            broken = True
            
        max_ = np.max([leftsided_pe, rightsided_pe, central_pe])  # this means image has no PE
        if max_ < 0.5:
            broken = True
            
    if not broken and has_pe_image:
        if rv_lv_ratio_gte_1 > 0.5 and rv_lv_ratio_lt_1 > 0.5:
            if rv_lv_ratio_gte_1 > rv_lv_ratio_lt_1:
                rv_lv_ratio_lt_1 = 0.5 - eps
            else:
                rv_lv_ratio_gte_1 = 0.5 - eps
        
        min_ = np.min([chronic_pe, acute_and_chronic_pe])  # almost never occurs
        if min_ > 0.5:
            if chronic_pe > acute_and_chronic_pe:
                acute_and_chronic_pe = 0.5 - eps
            else:
                chronic_pe = 0.5 - eps
        
    if broken:
        image_pred = np.clip(image_pred, 0, 0.5 - eps)
        assert image_pred.max() < 0.5
        
    has_pe_image = image_pred.max() > 0.5
    
    if not has_pe_image:   
        if indeterminate > 0.5 and negative_exam_for_pe > 0.5:
            if indeterminate > negative_exam_for_pe:
                negative_exam_for_pe = 0.5 - eps
            else:
                indeterminate = 0.5 - eps
        elif indeterminate < 0.5 and negative_exam_for_pe < 0.5:
            if indeterminate > negative_exam_for_pe:
                indeterminate = 0.5 + eps
            else:
                negative_exam_for_pe = 0.5 + eps
        
        (rv_lv_ratio_gte_1, rv_lv_ratio_lt_1, leftsided_pe, 
         chronic_pe, rightsided_pe, acute_and_chronic_pe, central_pe) = np.clip(
            exam_pred[[1, 2, 3, 4, 5, 6, 7]],
            0, 
            0.5 - eps
        )
        
    return np.array([
        negative_exam_for_pe,
        rv_lv_ratio_gte_1, 
        rv_lv_ratio_lt_1, 
        leftsided_pe, 
        chronic_pe,
        rightsided_pe,
        acute_and_chronic_pe,
        central_pe,
        indeterminate, 
    ]), image_pred

In [ ]:
def check_consistency(sub, test):
    
    '''
    Checks label consistency and returns the errors
    
    Args:
    sub   = submission dataframe (pandas)
    test  = test.csv dataframe (pandas)
    '''
    
    # EXAM LEVEL
    for i in test['StudyInstanceUID'].unique():
        df_tmp = sub.loc[sub.id.str.contains(i, regex = False)].reset_index(drop = True)
        df_tmp['StudyInstanceUID'] = df_tmp['id'].str.split('_').str[0]
        df_tmp['label_type']       = df_tmp['id'].str.split('_').str[1:].apply(lambda x: '_'.join(x))
        del df_tmp['id']
        if i == test['StudyInstanceUID'].unique()[0]:
            df = df_tmp.copy()
        else:
            df = pd.concat([df, df_tmp], axis = 0)
    df_exam = df.pivot(index = 'StudyInstanceUID', columns = 'label_type', values = 'label')
    
    # IMAGE LEVEL
    df_image = sub.loc[sub.id.isin(test.SOPInstanceUID)].reset_index(drop = True)
    df_image = df_image.merge(test, how = 'left', left_on = 'id', right_on = 'SOPInstanceUID')
    df_image.rename(columns = {"label": "pe_present_on_image"}, inplace = True)
    del df_image['id']
    
    # MERGER
    df = df_exam.merge(df_image, how = 'left', on = 'StudyInstanceUID')
    ids    = ['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']
    labels = [c for c in df.columns if c not in ids]
    df = df[ids + labels]
    
    # SPLIT NEGATIVE AND POSITIVE EXAMS
    df['positive_images_in_exam'] = df['StudyInstanceUID'].map(df.groupby(['StudyInstanceUID']).pe_present_on_image.max())
    df_pos = df.loc[df.positive_images_in_exam >  0.5]
    df_neg = df.loc[df.positive_images_in_exam <= 0.5]
    
    # CHECKING CONSISTENCY OF POSITIVE EXAM LABELS
    rule1a = df_pos.loc[((df_pos.rv_lv_ratio_lt_1  >  0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 >  0.5)) | 
                        ((df_pos.rv_lv_ratio_lt_1  <= 0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 <= 0.5))].reset_index(drop = True)
    rule1a['broken_rule'] = '1a'
    rule1b = df_pos.loc[(df_pos.central_pe    <= 0.5) & 
                        (df_pos.rightsided_pe <= 0.5) & 
                        (df_pos.leftsided_pe  <= 0.5)].reset_index(drop = True)
    rule1b['broken_rule'] = '1b'
    rule1c = df_pos.loc[(df_pos.acute_and_chronic_pe > 0.5) & 
                        (df_pos.chronic_pe           > 0.5)].reset_index(drop = True)
    rule1c['broken_rule'] = '1c'
    rule1d = df_pos.loc[(df_pos.indeterminate        > 0.5) | 
                        (df_pos.negative_exam_for_pe > 0.5)].reset_index(drop = True)
    rule1d['broken_rule'] = '1d'

    # CHECKING CONSISTENCY OF NEGATIVE EXAM LABELS
    rule2a = df_neg.loc[((df_neg.indeterminate        >  0.5)  & 
                         (df_neg.negative_exam_for_pe >  0.5)) | 
                        ((df_neg.indeterminate        <= 0.5)  & 
                         (df_neg.negative_exam_for_pe <= 0.5))].reset_index(drop = True)
    rule2a['broken_rule'] = '2a'
    rule2b = df_neg.loc[(df_neg.rv_lv_ratio_lt_1     > 0.5) | 
                        (df_neg.rv_lv_ratio_gte_1    > 0.5) |
                        (df_neg.central_pe           > 0.5) | 
                        (df_neg.rightsided_pe        > 0.5) | 
                        (df_neg.leftsided_pe         > 0.5) |
                        (df_neg.acute_and_chronic_pe > 0.5) | 
                        (df_neg.chronic_pe           > 0.5)].reset_index(drop = True)
    rule2b['broken_rule'] = '2b'
    
    # MERGING INCONSISTENT PREDICTIONS
    errors = pd.concat([rule1a, rule1b, rule1c, rule1d, rule2a, rule2b], axis = 0)
    
    # OUTPUT
    print('Found', len(errors), 'inconsistent predictions')
    return errors

# Inference

In [ ]:
from torch.utils.data import DataLoader

def extract_features(models, dataset, batch_size=32):
    fts = np.empty((0, models[0].nb_ft))
    zs = np.empty(0)
    preds = np.empty(0)

    loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=1,
        drop_last=False,
        pin_memory=True
    )
    
    with torch.no_grad():
        for x, z in loader:
            fts_tmp = []
            preds_tmp = []
            for model in models:
                pred, ft = model(x.cuda())
                preds_tmp.append(torch.sigmoid(pred).detach().cpu().numpy())
                fts_tmp.append(ft.detach().cpu().numpy())
                
            preds = np.concatenate([preds, np.mean(preds_tmp, 0)])
            fts = np.concatenate([fts, np.mean(fts_tmp, 0)])
            zs = np.concatenate([zs, z.numpy()])

    order = np.argsort(zs)
    fts = fts[order]
    preds = preds[order]
    
    return preds, fts, order

In [ ]:
def inference_second_lvl(model, ft):
    with torch.no_grad():
        x = torch.from_numpy(ft).unsqueeze(0).cuda().float()
        logits_exam, logits_img = model(x)
        
        pred_exam = torch.sigmoid(logits_exam).detach().cpu().squeeze().numpy()
        pred_img = torch.sigmoid(logits_img).detach().cpu().squeeze().numpy()
        
    return pred_exam, pred_img

In [ ]:
def pred_to_sub(unique_df, df, pred_exams, pred_imgs, orders):
    df_dic = {}
    for i, study in enumerate(unique_df['StudyInstanceUID'].values):
        reverse_order = np.argsort(orders[i])
        for t, target in enumerate(EXAM_TARGETS):
            name = f'{study}_{target}'
            label = pred_exams[i][t]
            df_dic[name] = label
        
        df_patient = df[df['StudyInstanceUID'] == study]
        for s, sop in enumerate(df_patient['SOPInstanceUID'].values):
            df_dic[sop] = pred_imgs[i][reverse_order[s]]
    
    sub = pd.DataFrame.from_dict(df_dic, orient='index', columns=['label']).reset_index()
    return sub.rename({'index': 'id'}, axis=1)

# Main

## Paths

In [ ]:
df_test = pd.read_csv(DATA_PATH + "test.csv")

In [ ]:
paths_test = [DATA_PATH + f"test/{study}/{series}/" for study, series in df_test[['StudyInstanceUID', 'SeriesInstanceUID']].values]
df_test['path'] = paths_test

In [ ]:
unique_df_test = df_test[['path', 'StudyInstanceUID', 'SeriesInstanceUID']].drop_duplicates().reset_index(drop=True)

## Pretrained models

In [ ]:
CP_PATH = "../input/peweights/"

In [ ]:
efficientnets = [f"efficientnet-b3__{i}.pt" for i in range(5)]

rnns_1 = [f'rnn_2_{i}.pt' for i in range(5)]
rnns_2 = [f'rnn_3_{i}.pt' for i in range(5)]

In [ ]:
first_lvl_models = []
    
for weights in efficientnets:
    model = define_model('efficientnet-b3').cuda().eval()
    model = load_model_weights(model, CP_PATH + weights)
    first_lvl_models.append(model)

In [ ]:
second_lvl_models = []

for weights in rnns_1:
    model = RNNModel(
        ft_dim=1536, 
        lstm_dim=64,
        use_msd=True,
    ).cuda().eval()
    
    model = load_model_weights(model, CP_PATH + weights)
    second_lvl_models.append(model)
    
# for weights in rnns_2:
#     model = RNNModel(
#         ft_dim=1536, 
#         lstm_dim=256,
#         use_msd=True,
#     ).cuda().eval()
    
#     model = load_model_weights(model, CP_PATH + weights)
#     second_lvl_models.append(model)

## Inference

In [ ]:
IS_PUBLIC = len(unique_df_test) <= 650
DO_PUBLIC = True
DO_PRIVATE = True

DO_INFERENCE = (IS_PUBLIC and DO_PUBLIC) or (not IS_PUBLIC and DO_PRIVATE)

In [ ]:
if DO_INFERENCE:
    all_pred_exams = []
    all_pred_imgs = []
    orders = []

    for path, study, series in tqdm(unique_df_test.values):
        df_patient = df_test[df_test['StudyInstanceUID'] == study].reset_index(drop=True)

        dataset = PatientDataset(df_patient, path, normalizer())

#         print('First level inference. \n')

        preds_1, fts, order = extract_features(first_lvl_models, dataset, batch_size=16)
        orders.append(order)

#         print('Second level inference. \n')

        pred_exams = []
        pred_imgs = []
        for model in second_lvl_models:
            pred_exam, pred_img = inference_second_lvl(model, fts)
            pred_exams.append(pred_exam)
            pred_imgs.append(pred_img)

        pred_exams = np.mean(pred_exams, 0)
        pred_imgs = np.mean(pred_imgs, 0)
        
        pred_exams, pred_imgs = post_process(pred_exams, pred_imgs, eps=1e-6)

        all_pred_exams.append(pred_exams)
        all_pred_imgs.append(pred_imgs)
        
    sub = pred_to_sub(unique_df_test, df_test, all_pred_exams, all_pred_imgs, orders)
else:
    sub = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [ ]:
check_consistency(sub, df_test)

In [ ]:
sub.to_csv('submission.csv', index=False)
sub